In [1]:
# import some common Python packages:
import altair as alt
import numpy as np
import pandas as pd

alt.data_transformers.enable(max_rows=20000)
pd.set_option('display.max_rows', 100)
pd.set_option('plotting.backend', 'altair')

In [2]:
data = pd.read_csv('gemliste_knz - 2022-01-27.csv', delimiter=';', skiprows=2, skipfooter=1, engine='python')
data.columns = ['gkz', 'gname', 'gcode', 'status', 'plz_primary', 'plz_additional']
data

,gkz,gname,gcode,status,plz_primary,plz_additional
0,10101,Eisenstadt,10101,SR,7000,NaN
1,10201,Rust,10201,SR,7071,NaN
2,10301,Breitenbrunn am Neusiedler See,10301,M,7091,NaN
3,10302,Donnerskirchen,10302,M,7082,NaN
4,10303,Großhöflein,10303,M,7051,NaN
...,...,...,...,...,...,...
2110,90001,Wien,91901,SR,1190,1090
2111,90001,Wien,92001,SR,1200,NaN
2112,90001,Wien,92101,SR,1210,NaN
2113,90001,Wien,92201,SR,1220,1210


In [3]:
primary = data.copy()
primary['plz'] = primary.plz_primary
primary['priority'] = primary.status.map({'SR': 3, 'ST': 2, 'M': 1}).fillna(0) + 100
primary

,gkz,gname,gcode,status,plz_primary,plz_additional,plz,priority
0,10101,Eisenstadt,10101,SR,7000,NaN,7000,103.0
1,10201,Rust,10201,SR,7071,NaN,7071,103.0
2,10301,Breitenbrunn am Neusiedler See,10301,M,7091,NaN,7091,101.0
3,10302,Donnerskirchen,10302,M,7082,NaN,7082,101.0
4,10303,Großhöflein,10303,M,7051,NaN,7051,101.0
...,...,...,...,...,...,...,...,...
2110,90001,Wien,91901,SR,1190,1090,1190,103.0
2111,90001,Wien,92001,SR,1200,NaN,1200,103.0
2112,90001,Wien,92101,SR,1210,NaN,1210,103.0
2113,90001,Wien,92201,SR,1220,1210,1220,103.0


In [4]:
additional = data.copy()
additional = additional.loc[~additional.plz_additional.isna(), :]
additional['plz'] = additional.plz_additional.str.split(' ')
additional['priority'] = additional.status.map({'SR': 3, 'ST': 2, 'M': 1}).fillna(0)
additional = additional.explode('plz')
additional['plz'] = additional['plz'].astype(int)
additional

,gkz,gname,gcode,status,plz_primary,plz_additional,plz,priority
5,10304,Hornstein,10304,M,7053,2491,2491,1.0
17,10316,Steinbrunn,10316,M,7035,2491,2491,1.0
23,10322,Zillingtal,10322,NaN,7034,7033 7035,7033,0.0
23,10322,Zillingtal,10322,NaN,7034,7033 7035,7035,0.0
26,10402,Burgauberg-Neudauberg,10402,NaN,8291,7574,7574,0.0
...,...,...,...,...,...,...,...,...
2104,90001,Wien,91301,SR,1130,1120,1120,3.0
2105,90001,Wien,91401,SR,1140,1130,1130,3.0
2109,90001,Wien,91801,SR,1180,1190,1190,3.0
2110,90001,Wien,91901,SR,1190,1090,1090,3.0


In [5]:
mapping = pd.concat([primary, additional]).reset_index(drop=True)

# For each PLZ get max priority entry
mapping = mapping.loc[mapping.groupby(['plz'])['priority'].idxmax().values, :].reset_index(drop=True)

mapping

,gkz,gname,gcode,status,plz_primary,plz_additional,plz,priority
0,90001,Wien,90101,SR,1010,NaN,1010,103.0
1,90001,Wien,90201,SR,1020,NaN,1020,103.0
2,90001,Wien,90301,SR,1030,NaN,1030,103.0
3,90001,Wien,90401,SR,1040,1010,1040,103.0
4,90001,Wien,90501,SR,1050,NaN,1050,103.0
...,...,...,...,...,...,...,...,...
2219,70723,Prägraten am Großvenediger,70723,NaN,9974,NaN,9974,100.0
2220,70712,Kals am Großglockner,70712,NaN,9981,NaN,9981,100.0
2221,70719,Nußdorf-Debant,70719,M,9990,NaN,9990,101.0
2222,70707,Dölsach,70707,NaN,9991,NaN,9991,100.0


In [6]:
def get_plz_gcode_mapping(gemliste_file='gemliste_knz - 2022-01-27.csv', minimal=True):
    data = pd.read_csv(gemliste_file, delimiter=';', skiprows=2, skipfooter=1, engine='python')
    data.columns = ['gkz', 'gname', 'gcode', 'status', 'plz_primary', 'plz_additional']
    
    # entries where PLZ occurrs in primary column
    primary = data.copy()
    primary['plz'] = primary.plz_primary
    primary['priority'] = primary.status.map({'SR': 3, 'ST': 2, 'M': 1}).fillna(0) + 100
    primary.iloc[:50]
    
    # entries where PLZ occurrs in additional column
    additional = data.copy()
    additional = additional.loc[~additional.plz_additional.isna(), :]
    additional['plz'] = additional.plz_additional.str.split(' ')
    additional['priority'] = additional.status.map({'SR': 3, 'ST': 2, 'M': 1}).fillna(0)
    additional = additional.explode('plz')
    additional['plz'] = additional['plz'].astype(int)
    
    mapping = pd.concat([primary, additional]).reset_index(drop=True)

    # For each PLZ get max priority entry
    mapping = mapping.loc[mapping.groupby(['plz'])['priority'].idxmax().values, :].reset_index(drop=True)
    
    if minimal:
        mapping = mapping.loc[:, ['plz', 'gcode']]

    return mapping

get_plz_gcode_mapping()

,plz,gcode
0,1010,90101
1,1020,90201
2,1030,90301
3,1040,90401
4,1050,90501
...,...,...
2219,9974,70723
2220,9981,70712
2221,9990,70719
2222,9991,70707


In [7]:
%timeit get_plz_gcode_mapping()

212 ms ± 37.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
